In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [23]:
features_df = pd.read_csv('/content/Dataset_engineered.csv')

In [6]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13414 entries, 0 to 13413
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   age                                 13414 non-null  float64
 1   num_subscription_pauses             13414 non-null  int64  
 2   weekly_hours                        13414 non-null  float64
 3   average_session_length              13414 non-null  float64
 4   song_skip_rate                      13414 non-null  float64
 5   weekly_songs_played                 13414 non-null  int64  
 6   weekly_unique_songs                 13414 non-null  int64  
 7   num_favorite_artists                13414 non-null  int64  
 8   num_platform_friends                13414 non-null  int64  
 9   num_playlists_created               13414 non-null  int64  
 10  num_shared_playlists                13414 non-null  int64  
 11  notifications_clicked               13414

In [24]:
X = features_df.drop(columns=['churned'])
y = features_df['churned']

In [25]:
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [28]:
def tune_and_evaluate(model, param_grid, X_train, y_train, X_test, y_test, model_name):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Best Parameters: {grid_search.best_params_}")
    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Classification Report:\n{classification_report(y_test, y_pred)}")

    if hasattr(best_model, 'predict_proba'):
        y_pred_proba = best_model.predict_proba(X_test)
        if y_pred_proba.shape[1] > 2:  # Multi-class case
            roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
        else:  # Binary case
            roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1] if y_pred_proba.shape[1] > 1 else y_pred_proba.ravel())
        print(f"{model_name} ROC-AUC Score: {roc_auc:.4f}")

    print("-" * 50)
    return accuracy



In [29]:
logistic_params = {'C': [0.01, 0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}
logistic_acc = tune_and_evaluate(LogisticRegression(max_iter=500), logistic_params, X_train_scaled, y_train, X_test_scaled, y_test, "Logistic Regression")


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Logistic Regression Best Parameters: {'C': 10, 'solver': 'lbfgs'}
Logistic Regression Accuracy: 0.7930
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.79      0.79     12163
           1       0.80      0.80      0.80     12837

    accuracy                           0.79     25000
   macro avg       0.79      0.79      0.79     25000
weighted avg       0.79      0.79      0.79     25000

Logistic Regression ROC-AUC Score: 0.8805
--------------------------------------------------


In [30]:
dt_params = {'max_depth': [5, 10, 20, None], 'min_samples_split': [2, 5, 10]}
dt_acc = tune_and_evaluate(DecisionTreeClassifier(), dt_params, X_train, y_train, X_test, y_test, "Decision Tree")

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Decision Tree Best Parameters: {'max_depth': 10, 'min_samples_split': 2}
Decision Tree Accuracy: 0.8226
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.82      0.82     12163
           1       0.83      0.83      0.83     12837

    accuracy                           0.82     25000
   macro avg       0.82      0.82      0.82     25000
weighted avg       0.82      0.82      0.82     25000

Decision Tree ROC-AUC Score: 0.9064
--------------------------------------------------


In [31]:
rf_params = {'n_estimators': [50, 100, 200], 'max_depth': [10, 20, None]}
rf_acc = tune_and_evaluate(RandomForestClassifier(), rf_params, X_train, y_train, X_test, y_test, "Random Forest")


Fitting 5 folds for each of 9 candidates, totalling 45 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Random Forest Best Parameters: {'max_depth': 20, 'n_estimators': 200}
Random Forest Accuracy: 0.8281
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.82      0.82     12163
           1       0.83      0.84      0.83     12837

    accuracy                           0.83     25000
   macro avg       0.83      0.83      0.83     25000
weighted avg       0.83      0.83      0.83     25000

Random Forest ROC-AUC Score: 0.9149
--------------------------------------------------


In [32]:
print("\nModel Accuracy Comparison:")
print(f"Logistic Regression: {logistic_acc:.4f}")
print(f"Decision Tree: {dt_acc:.4f}")
print(f"Random Forest: {rf_acc:.4f}")


Model Accuracy Comparison:
Logistic Regression: 0.7930
Decision Tree: 0.8226
Random Forest: 0.8281
